In [1]:
# imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = 'plotly'

In [2]:
# load data
score_ARIMA_7 = pd.read_pickle("results/f8/ARIMA_result_7.pkl").set_index("store")
score_PROPHET_7 = pd.read_pickle("results/f8/PROPHET_result_7.pkl").set_index("store")
score_RF_7 = pd.read_pickle('results/grouping/result_RF_base.pkl').set_index("store")

score_ALL_7 = score_ARIMA_7.join(score_PROPHET_7).join(score_RF_7)[['mape_ARIMA','mape_PROPHET','mape_RF']].reset_index()
score_ALL_7['store'] = score_ALL_7['store'].map(lambda x:x[6:])
score_ALL_7.head(3)



,store,mape_ARIMA,mape_PROPHET,mape_RF
0,307222,0.2810,0.234,0.0480
1,307244,0.3455,0.309,0.0625
2,307248,0.3630,0.239,0.0585


In [3]:
data = {}
models = ["ARIMA", "PROPHET", "RF"]
groups = ["base","store_level", "store_segment", "cluster"] 
for model in models:
    for group in groups:
        if group == "base":
            data[model+"_"+group]= score_ALL_7[['store','mape_'+model]].rename({'mape_'+model:'mape'},axis=1)
        else:
            data[model+"_"+group] = pd.read_pickle("results/grouping/result_"+model+"_"+group+".pkl")



In [4]:
df = pd.DataFrame(columns=['store','mape','group','model'])
for model in models:    
    for group in groups:
        df=pd.concat([
                    df,
                    pd.DataFrame({
                        'store':data[model+"_"+group]['store'],
                        'mape':data[model+"_"+group]['mape'],
                        'group':group,
                        'model':model
                    })
        ])

In [5]:
fig = px.box(df, x="model", y="mape", color="group")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

In [7]:
data['RF_store_level']

,store_level,store,mape
0,A,307222,0.08625
1,A,307244,0.14225
2,A,328165,0.174
3,A,349920,0.129
4,A,349924,0.1205
5,A,349958,0.125
6,A,349980,0.0895
7,A,350018,0.204
8,A,350040,0.16275
9,A,350060,0.095
